In [ ]:
#Importamos las bibliotecas necesarias

import spotipy
import pandas as pd
import warnings
#Filtramos advertencias
from tqdm import tqdm

In [ ]:
# Nos da avisos de posibles errores
warnings.filterwarnings('ignore')

In [ ]:
#Importamos las credenciales de acceso a la API Spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Colocamos las credenciales

CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
#Fechas escogidas:

año_inicio = 2013
año_fin = 2017

In [ ]:
#Géneros escogidos: Flamencopop, Reggae, Ska y Kpop

In [ ]:
#Creamos una función para extraer los datos que nos interesan de cada género

def get_tracks_by_genre(genre):
    
    track_data = []

    # Buscamos 1000 canciones del género especificado, poniendo 50 por página (el máximo de la API).
    for offset in range(0, 1000, 50):
        results = sp.search(q=f'genre:{genre} year:{año_inicio}-{año_fin}', type='track', limit=50, offset=offset)
 
    # Extraemos los IDs de las canciones.
        track_ids = [track['id'] for track in results['tracks']['items']]
 
    # Obtenemos los detalles de las canciones.
        tracks_info = sp.tracks(tracks=track_ids)
 
    # Obtenemos los datos de cada canción y se los añadimos a la lista.
        for track in tracks_info['tracks']:
            track_data.append({
                'genre': genre,
                'name': track['name'],
                'artists_name': ', '.join([artist['name'] for artist in track['artists']]),  # Si hay varios artistas
                'album_name': track['album']['name'],
                'album_release_date': track['album']['release_date'],
                'duration_ms': track['duration_ms'],
                'popularity': track['popularity'],
                'track_number': track['track_number'],
                'uri': track['uri']
            })
 
    # Convertimos la lista en un df de pandas.
    
    df = pd.DataFrame(track_data)
 
    return df

In [ ]:
#Usamos la función con cada género

genre1 = "k_pop"
df_k_pop = get_tracks_by_genre(genre1)
df_k_pop

In [ ]:
#Convertimos los df resultantes de cada género en un archivo csv y uno excel (para verlo mas claro)

df_k_pop.to_csv('df_k_pop.csv', index=False)
df_k_pop.to_excel('df_k_pop.xlsx', index=False)

In [ ]:
# Dividimos los nombres por coma y aplanamos la lista
artistas = df_k_pop['artists_name'].str.split(',').explode().str.strip()

# Obtenemos artistas únicos para sacar los artistas en la API de Last.fm
artistas_unicos_kpop = set(artistas)

# Imprimimos el resultado iterando por el set
print("Artistas únicos encontrados:")
for artista in sorted(artistas_unicos_kpop):
    print(artista)
    

In [ ]:
genre2 = "ska"
df_ska = get_tracks_by_genre(genre2)
df_ska

In [ ]:
df_ska.to_csv('df_ska.csv', index=False)
df_ska.to_excel('df_ska.xlsx', index=False)

In [ ]:
artistas = df_ska['artists_name'].str.split(',').explode().str.strip()

artistas_unicos_ska = set(artistas)

print("Artistas únicos encontrados:")
for artista in sorted(artistas_unicos_ska):
    print(artista)

In [ ]:
genre3 = "reggae"
df_reggae = get_tracks_by_genre(genre3)
df_reggae

In [ ]:
df_reggae.to_csv('df_reggae.csv', index=False)
df_reggae.to_excel('df_reggae.xlsx', index=False)

In [ ]:
artistas = df_reggae['artists_name'].str.split(',').explode().str.strip()

artistas_unicos_reggae = set(artistas)

print("Artistas únicos encontrados:")
for artista in sorted(artistas_unicos_reggae):
    print(artista)

In [ ]:
genre4 = "flamenco_pop"
df_flamenco_pop = get_tracks_by_genre(genre4)
df_flamenco_pop

In [ ]:
df_flamenco_pop.to_csv('df_flamenco_pop.csv', index=False)
df_flamenco_pop.to_excel('df_flamenco_pop.xlsx', index=False)

In [ ]:
artistas = df_flamenco_pop['artists_name'].str.split(',').explode().str.strip()

artistas_unicos_flamencopop = set(artistas)

print("Artistas únicos encontrados:")
for artista in sorted(artistas_unicos_flamencopop):
    print(artista)

In [ ]:
#Imprimimos la largura de artistas únicos para compararla con la largura de los excel

print(len(artistas_unicos_kpop))
print(len(artistas_unicos_ska))
print(len(artistas_unicos_reggae))
print(len(artistas_unicos_flamencopop))


In [ ]:
#Convertimos la lista artistas únicos en un df de artistas por género

df_artistas1 = pd.DataFrame(artistas_unicos_kpop)
df_artistas2 = pd.DataFrame(artistas_unicos_ska)
df_artistas3 = pd.DataFrame(artistas_unicos_reggae)
df_artistas4 = pd.DataFrame(artistas_unicos_flamencopop)

In [ ]:
#Creamos archivos csv con esos df

df_artistas1.to_csv('df_k_pop_artistas.csv', index=False)
df_artistas2.to_csv('df_ska_artistas.csv', index=False)
df_artistas3.to_csv('df_reggae_artistas.csv', index=False)
df_artistas4.to_csv('df_flamenco_pop_artistas.csv', index=False)

In [ ]:
#Importamos las bibliotecas necesarias

import requests
import time  # Para manejar el límite de velocidad de la API

# Obtenemos la API Key de last.fm
API_KEY = 'd97c59010214fb89dd7e101407646224'  # Reemplazamos con nuestro API Key válida de Last.fm.
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'  # URL base de la API de Last.fm, siempre es la misma

#Creamos 3 funciones para extraer los datos que nos interesan de cada artista en Last.fm 
    # 1 Para extraer la info de artistas

def get_artist_info(artist_name, api_key):
    """
    Obtiene información de un artista desde la API de Last.fm.

    Args:
        artist_name (str): El nombre del artista.
        api_key (str): Tu clave de API de Last.fm.

    Returns:
        dict: Un diccionario con la información del artista, o None si hay un error.
    """
    params = {
        'method': 'artist.getinfo',  # Método de la API para obtener información del artista
        'artist': artist_name,       # Nombre del artista a buscar
        'api_key': api_key,         # Clave de la API
        'format': 'json'            # Formato de la respuesta (JSON)
    }
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP malos (4xx o 5xx)
        return response.json()       # Convierte la respuesta JSON a un diccionario de Python
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener información para {artist_name}: {e}")
        return None  # Devuelve None en caso de error

    # 2 Para crear un diccionario de cada respuesta

def create_artist_object(artist_data, artist_name):
    """
    Crea un diccionario con la información relevante del artista a partir de la respuesta de la API.

    Args:
        artist_data (dict): La respuesta JSON de la API de Last.fm.
        artist_name (str): El nombre del artista.

    Returns:
        dict: Un diccionario con la información del artista formateada.
    """
    if artist_data and 'artist' in artist_data:
        # Extrae la lista de artistas similares (maneja el caso de que falten datos)
        similar_artists = artist_data['artist'].get('similar', {}).get('artist', [])
        similar_names = [artist.get('name') for artist in similar_artists]  # Obtiene solo los nombres

        return {
            'artist': artist_name,
            'Biography': artist_data['artist'].get('bio', {}).get('summary', 'Sin datos'),
            'Listeners': artist_data['artist'].get('stats', {}).get('listeners', 'Sin datos'),
            'Playcount': artist_data['artist'].get('stats', {}).get('playcount', 'Sin datos'),
            'Similar Artists': similar_names  # Lista de nombres de artistas similares
        }
    else:
        return None  # Devuelve None si no hay datos del artista

    # 3 Para crear un df con la info de cada artista

def process_artists_from_dataframe(artists_df, api_key, artist_column_name=0):
    """
    Obtiene y procesa información de artistas desde un DataFrame de pandas.

    Args:
        artists_df (pd.DataFrame): DataFrame que contiene los nombres de los artistas.
        api_key (str): Tu clave de API de Last.fm.
        artist_column_name (str o int, opcional): El nombre o el índice de la columna
                                                  que contiene los nombres de los artistas.
                                                  Por defecto, es 0 (la primera columna).

    Returns:
        pd.DataFrame: DataFrame con la información de los artistas.
    """
    artists_data = []
    for artist in artists_df[artist_column_name]:  # Itera sobre la columna de artistas
        artist_info = get_artist_info(artist, api_key)  # Obtiene información del artista
        if artist_info:
            artista = create_artist_object(artist_info, artist)  # Crea el objeto artista
            if artista:
                artists_data.append(artista)  # Agrega la información a la lista
        time.sleep(0.1)  # Aplica un retraso para evitar el límite de velocidad de la API
    return pd.DataFrame(artists_data)  # Convierte la lista de diccionarios a un DataFrame


# Bloque de ejecución principal (solo se ejecuta si el script se ejecuta directamente)
if __name__ == '__main__':
    
    df_artists_info_kpop = process_artists_from_dataframe(df_artistas1, API_KEY, artist_column_name=0)
    df_artists_info_ska = process_artists_from_dataframe(df_artistas2, API_KEY, artist_column_name=0)
    df_artists_info_reggae = process_artists_from_dataframe(df_artistas3, API_KEY, artist_column_name=0)
    df_artists_info_flamenco = process_artists_from_dataframe(df_artistas4, API_KEY, artist_column_name=0)

    # Imprimimos los resultados 
    print("Información de Artistas K-pop:\n", df_artists_info_kpop)
    print("\nInformación de Artistas Ska:\n", df_artists_info_ska)
    print("\nInformación de Artistas Reggae:\n", df_artists_info_reggae)
    print("\nInformación de Artistas Flamenco-Pop:\n", df_artists_info_flamenco)

In [ ]:
# Convertimos a csv los df de artistas

df_artists_info_kpop.to_csv("kpop_artistas_info.csv", index=False)
df_artists_info_ska.to_csv("ska_artistas_info.csv", index=False)
df_artists_info_reggae.to_csv("reggae_artistas_info.csv", index=False)
df_artists_info_flamenco.to_csv("flamenco_pop_artistas_info.csv", index=False)

In [ ]:
# Convertimos a Excel los df de artistas

df_artists_info_kpop.to_excel("kpop_artistas_info.xlsx", index=False)
df_artists_info_ska.to_excel("ska_artistas_info.xlsx", index=False)
df_artists_info_reggae.to_excel("reggae_artistas_info.xlsx", index=False)
df_artists_info_flamenco.to_excel("flamenco_pop_artistas_info.xlsx", index=False)

In [3]:
# Importamos las bibliotecas necesarias para la conexión con SQL

import mysql.connector
from mysql.connector import errorcode 

In [22]:
# Realizamos la conexión y creamos el cursor

try:
    cnx = mysql.connector.connect(
        user = 'root',
        password = 'AlumnaAdalab',
        host = '127.0.0.1',
        port = 3306
    )
    print('Conexión existosa a SQL')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('Algo está mal con tu usuario o contraseña')
    else:
        print(f'Error:{err}')

cursor = cnx.cursor()

Conexión existosa a SQL


In [ ]:
# Creamos el esquema en sql y lo empezamos a usar (esto solo se ejecuta una vez)

cursor.execute('CREATE DATABASE proyecto_2_MusicStream')
cursor.execute('USE proyecto_2_MusicStream')

# Cerramos la conexión con SQL

cnx.close()

In [1]:
# Importamos desde aquí el pandas para seguir trabajando desde este punto sin necesidad de ejecutar el resto de código

import pandas as pd

In [28]:
# A partir de aquí nos conectamos directamente al esquema creado

try:
    cnx = mysql.connector.connect(
        user = 'root',
        password = 'AlumnaAdalab',
        host = '127.0.0.1',
        port = 3306,
        database = 'proyecto_2_MusicStream'
    )
    print('Conexión existosa a la base de datos')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('Algo está mal con tu usuario o contraseña')
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print('La base de datos a la que intentas conectar no existe')
    else:
        print(f'Error:{err}')

# Creamos el cursor

cursor = cnx.cursor()

Conexión existosa a la base de datos


In [6]:
# Para insertar los registros en la base de datos por cada género:

# Cargamos los csv's ,de ambas API's, creados anteriormente

df_reggae = pd.read_csv("df_reggae.csv")
df_reggae_artistas = pd.read_csv("reggae_artistas_info.csv")

# Insertamos en tabla 'reggae' los datos del df Spotify

for _, fila in df_reggae.iterrows():
    cursor.execute("""
        INSERT INTO reggae 
        (genero, cancion, nombre_artista, album, año, duracion, popularidad, numero_pista, spotify_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        fila['genre'], fila['name'], fila['artists_name'], fila['album_name'],
        fila['album_release_date'], int(fila['duration_ms']), int(fila['popularity']),
        fila['track_number'], fila['uri']
    ))

# Confirmamos los cambios en la base de datos

cnx.commit()

# Insertamos en tabla 'reggae_artistas' los datos del df Last.fm

for _, fila in df_reggae_artistas.iterrows():
    cursor.execute("""
        INSERT INTO reggae_artistas 
        (artista, biografia, oyentes, reproducciones, artistas_similares)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        fila['artist'], fila['Biography'], int(fila['Listeners']),
        int(fila['Playcount']), fila['Similar Artists']
    ))

# Confirmamos los cambios en la base de datos

cnx.commit()


print("✅ FASE 1: Datos insertados en reggae y reggae_artistas.")

✅ FASE 1: Datos insertados en reggae y reggae_artistas.


In [7]:
# Tenemos que separar los artistas de la tabla derivada de Spotify, porque hay canciones colaborativas
# y salen todos en el mismo registro, y así obtenemos los datos únicos:

    # 1 Cargamos los datos de la tabla en un df
df_reggae = pd.read_sql("SELECT * FROM reggae", cnx)

    # 2 Procesamos los artistas
    # 3 Creamos un df donde cada artista se asocia con su canción

artistas_y_canciones_reggae = []

for _, fila in df_reggae.iterrows():
    artistas_reggae = fila['nombre_artista'].split(',')  # Separamos los artistas por coma
    for artista in artistas_reggae:
        artistas_y_canciones_reggae.append((fila['id_cancion'], artista.strip()))  # Limpiamos los espacios y lo asociamos a la canción

    # 4 Creamos un df con canciones y artistas

df_artistas_canciones_reggae = pd.DataFrame(artistas_y_canciones_reggae, columns=['id_cancion', 'artist'])

C:\Users\AIDA\AppData\Local\Temp\ipykernel_12532\3357808198.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reggae = pd.read_sql("SELECT * FROM reggae", cnx)


In [11]:
# Para insertar los artistas en la tabla reggae_artistas:

# Eliminamos duplicados

df_artistas_unicos_reggae = df_artistas_canciones_reggae['artist'].drop_duplicates()

# Insertamos los artistas únicos en su tabla 

for artista in df_artistas_unicos_reggae:
    cursor.execute("""
        INSERT INTO reggae_artistas (artista) 
        SELECT %s
        WHERE NOT EXISTS (SELECT 1 FROM reggae_artistas WHERE artista = %s)
    """, (artista, artista))

# Confirmamos los cambios en la base de datos

cnx.commit()

In [13]:
# Para obtener los IDs de los artistas:

# Obtenemos los id's de los artistas de sql

artistas_ids_reggae = {}
cursor.execute("SELECT id_artista, artista FROM reggae_artistas")
for fila in cursor.fetchall():
    artistas_ids_reggae[fila[1]] = fila[0]  # Asociamos el nombre del artista con su id

# Añadimos el id_artista al df

df_artistas_canciones_reggae['id_artista'] = df_artistas_canciones_reggae['artist'].map(artistas_ids_reggae)

In [14]:
# Para insertar las relaciones en la tabla intermedia de cada género (creadas en SQL):

# Insertamos las relaciones en su tabla intermedia 

for _, fila in df_artistas_canciones_reggae.iterrows():
    cursor.execute("""
        INSERT INTO canciones_artistas_reggae (id_cancion, id_artista) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE id_artista = id_artista
    """, (fila['id_cancion'], fila['id_artista']))

# Confirmamos los cambios en la base de datos

cnx.commit()

print("✅ Relaciones insertadas correctamente.")

✅ Relaciones insertadas correctamente.


In [15]:
# Para insertar los datos de ska en sus tablas primarias:

df_ska = pd.read_csv("df_ska.csv")
df_ska_artistas = pd.read_csv("ska_artistas_info.csv")

for _, fila in df_ska.iterrows():
    cursor.execute("""
        INSERT INTO ska 
        (genero, cancion, nombre_artista, album, año, duracion, popularidad, numero_pista, spotify_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        fila['genre'], fila['name'], fila['artists_name'], fila['album_name'],
        fila['album_release_date'], int(fila['duration_ms']), int(fila['popularity']),
        fila['track_number'], fila['uri']
    ))

cnx.commit()

for _, fila in df_ska_artistas.iterrows():
    cursor.execute("""
        INSERT INTO ska_artistas 
        (artista, biografia, oyentes, reproducciones, artistas_similares)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        fila['artist'], fila['Biography'], int(fila['Listeners']),
        int(fila['Playcount']), fila['Similar Artists']
    ))

cnx.commit()


print("✅ FASE 1: Datos insertados en ska y ska_artistas.")

✅ FASE 1: Datos insertados en ska y ska_artistas.


In [17]:
# # Para extraer los datos necesarios para la tabla intermedia e insertarlos en ska:

df_ska = pd.read_sql("SELECT * FROM ska", cnx)
    
artistas_y_canciones_ska = []

for _, fila in df_ska.iterrows():
    artistas_ska = fila['nombre_artista'].split(',')  
    for artista in artistas_ska:
        artistas_y_canciones_ska.append((fila['id_cancion'], artista.strip()))  

df_artistas_canciones_ska = pd.DataFrame(artistas_y_canciones_ska, columns=['id_cancion', 'artist'])


df_artistas_unicos_ska = df_artistas_canciones_ska['artist'].drop_duplicates()

for artista in df_artistas_unicos_ska:
    cursor.execute("""
        INSERT INTO ska_artistas (artista) 
        SELECT %s
        WHERE NOT EXISTS (SELECT 1 FROM ska_artistas WHERE artista = %s)
    """, (artista, artista))

cnx.commit()


artistas_ids_ska = {}
cursor.execute("SELECT id_artista, artista FROM ska_artistas")
for fila in cursor.fetchall():
    artistas_ids_ska[fila[1]] = fila[0]  

df_artistas_canciones_ska['id_artista'] = df_artistas_canciones_ska['artist'].map(artistas_ids_ska)


for _, fila in df_artistas_canciones_ska.iterrows():
    cursor.execute("""
        INSERT INTO canciones_artistas_ska (id_cancion, id_artista) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE id_artista = id_artista
    """, (fila['id_cancion'], fila['id_artista']))

cnx.commit()

print("✅ Relaciones insertadas correctamente.")

C:\Users\AIDA\AppData\Local\Temp\ipykernel_12532\2160625949.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ska = pd.read_sql("SELECT * FROM ska", cnx)


✅ Relaciones insertadas correctamente.


In [18]:
# Para insertar los datos de flamencopop en sus tablas primarias:

df_flamencopop = pd.read_csv("df_flamenco_pop.csv")
df_flamencopop_artistas = pd.read_csv("flamenco_pop_artistas_info.csv")

for _, fila in df_flamencopop.iterrows():
    cursor.execute("""
        INSERT INTO flamencopop 
        (genero, cancion, nombre_artista, album, año, duracion, popularidad, numero_pista, spotify_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        fila['genre'], fila['name'], fila['artists_name'], fila['album_name'],
        fila['album_release_date'], int(fila['duration_ms']), int(fila['popularity']),
        fila['track_number'], fila['uri']
    ))

cnx.commit()

for _, fila in df_flamencopop_artistas.iterrows():
    cursor.execute("""
        INSERT INTO flamencopop_artistas 
        (artista, biografia, oyentes, reproducciones, artistas_similares)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        fila['artist'], fila['Biography'], int(fila['Listeners']),
        int(fila['Playcount']), fila['Similar Artists']
    ))

cnx.commit()


print("✅ FASE 1: Datos insertados en flamencopop y flamencopop_artistas.")

✅ FASE 1: Datos insertados en flamencopop y flamencopop_artistas.


In [19]:
# Para extraer los datos necesarios para la tabla intermedia e insertarlos en flamencopop:

df_flamencopop = pd.read_sql("SELECT * FROM flamencopop", cnx)
    
artistas_y_canciones_flamencopop = []

for _, fila in df_flamencopop.iterrows():
    artistas_flamencopop = fila['nombre_artista'].split(',')  
    for artista in artistas_flamencopop:
        artistas_y_canciones_flamencopop.append((fila['id_cancion'], artista.strip()))  

df_artistas_canciones_flamencopop = pd.DataFrame(artistas_y_canciones_flamencopop, columns=['id_cancion', 'artist'])


df_artistas_unicos_flamencopop = df_artistas_canciones_flamencopop['artist'].drop_duplicates()

for artista in df_artistas_unicos_flamencopop:
    cursor.execute("""
        INSERT INTO flamencopop_artistas (artista) 
        SELECT %s
        WHERE NOT EXISTS (SELECT 1 FROM flamencopop_artistas WHERE artista = %s)
    """, (artista, artista))

cnx.commit()


artistas_ids_flamencopop = {}
cursor.execute("SELECT id_artista, artista FROM flamencopop_artistas")
for fila in cursor.fetchall():
    artistas_ids_flamencopop[fila[1]] = fila[0]  

df_artistas_canciones_flamencopop['id_artista'] = df_artistas_canciones_flamencopop['artist'].map(artistas_ids_flamencopop)


for _, fila in df_artistas_canciones_flamencopop.iterrows():
    cursor.execute("""
        INSERT INTO canciones_artistas_flamencopop (id_cancion, id_artista) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE id_artista = id_artista
    """, (fila['id_cancion'], fila['id_artista']))

cnx.commit()

print("✅ Relaciones insertadas correctamente.")

C:\Users\AIDA\AppData\Local\Temp\ipykernel_12532\4153856974.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_flamencopop = pd.read_sql("SELECT * FROM flamencopop", cnx)


✅ Relaciones insertadas correctamente.


In [29]:
# Para insertar los datos de kpop en sus tablas primarias:

df_kpop = pd.read_csv("df_k_pop.csv")
df_kpop_artistas = pd.read_csv("kpop_artistas_info.csv")

for _, fila in df_kpop.iterrows():
    cursor.execute("""
        INSERT INTO kpop 
        (genero, cancion, nombre_artista, album, año, duracion, popularidad, numero_pista, spotify_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        fila['genre'], fila['name'], fila['artists_name'], fila['album_name'],
        fila['album_release_date'], int(fila['duration_ms']), int(fila['popularity']),
        fila['track_number'], fila['uri']
    ))

cnx.commit()

for _, fila in df_kpop_artistas.iterrows():
    cursor.execute("""
        INSERT INTO kpop_artistas 
        (artista, biografia, oyentes, reproducciones, artistas_similares)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        fila['artist'], fila['Biography'], int(fila['Listeners']),
        int(fila['Playcount']), fila['Similar Artists']
    ))

cnx.commit()


print("✅ FASE 1: Datos insertados en kpop y kpop_artistas.")

✅ FASE 1: Datos insertados en kpop y kpop_artistas.


In [ ]:
# Para extraer los datos necesarios para la tabla intermedia e insertarlos en kpop:

df_kpop = pd.read_sql("SELECT * FROM kpop", cnx)
    
artistas_y_canciones_kpop = []

for _, fila in df_kpop.iterrows():
    if pd.isna(fila['id_cancion']) or pd.isna(fila['nombre_artista']):
        continue  # como nos da fallo al insertar los datos por registros Nan, saltamos si falta algún dato

    artistas_kpop = fila['nombre_artista'].split(',')
    for artista in artistas_kpop:
        artista_limpio = artista.strip()
        if artista_limpio.lower() == 'nan' or artista_limpio == '':
            continue  # evitamos artistas vacíos o mal formateados

        artistas_y_canciones_kpop.append((int(fila['id_cancion']), artista_limpio)) 


df_artistas_canciones_kpop = pd.DataFrame(artistas_y_canciones_kpop, columns=['id_cancion', 'artist'])

df_artistas_unicos_kpop = df_artistas_canciones_kpop['artist'].drop_duplicates()

for artista in df_artistas_unicos_kpop:
    cursor.execute("""
        INSERT INTO kpop_artistas (artista) 
        SELECT %s
        WHERE NOT EXISTS (SELECT 1 FROM kpop_artistas WHERE artista = %s)
    """, (artista, artista))

cnx.commit()

artistas_ids_kpop = {}
cursor.execute("SELECT id_artista, artista FROM kpop_artistas")
for fila in cursor.fetchall():
    artistas_ids_kpop[fila[1]] = fila[0]  

df_artistas_canciones_kpop['id_artista'] = df_artistas_canciones_kpop['artist'].map(artistas_ids_kpop)
df_artistas_canciones_kpop = df_artistas_canciones_kpop.where(pd.notnull(df_artistas_canciones_kpop), None)

# Eliminamos los registros que tienen nan craedos por map, antes de insertarlos en la tabla

df_artistas_canciones_kpop = df_artistas_canciones_kpop.dropna(subset=['id_artista'])

for _, fila in df_artistas_canciones_kpop.iterrows():
    cursor.execute("""
        INSERT INTO canciones_artistas_kpop (id_cancion, id_artista) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE id_artista = id_artista
    """, (fila['id_cancion'], fila['id_artista']))

cnx.commit()

print("✅ Relaciones insertadas correctamente.")

C:\Users\AIDA\AppData\Local\Temp\ipykernel_12532\917894883.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_kpop = pd.read_sql("SELECT * FROM kpop", cnx)


In [36]:
for artista in df_artistas_unicos_kpop:
    cursor.execute("""
        INSERT INTO kpop_artistas (artista) 
        SELECT %s
        WHERE NOT EXISTS (SELECT 1 FROM kpop_artistas WHERE artista = %s)
    """, (artista, artista))

cnx.commit()

artistas_ids_kpop = {}
cursor.execute("SELECT id_artista, artista FROM kpop_artistas")
for fila in cursor.fetchall():
    artistas_ids_kpop[fila[1]] = fila[0]  

df_artistas_canciones_kpop['id_artista'] = df_artistas_canciones_kpop['artist'].map(artistas_ids_kpop)
df_artistas_canciones_kpop = df_artistas_canciones_kpop.where(pd.notnull(df_artistas_canciones_kpop), None)

df_artistas_canciones_kpop = df_artistas_canciones_kpop.dropna(subset=['id_artista'])

for _, fila in df_artistas_canciones_kpop.iterrows():
    cursor.execute("""
        INSERT INTO canciones_artistas_kpop (id_cancion, id_artista) 
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE id_artista = id_artista
    """, (fila['id_cancion'], fila['id_artista']))

cnx.commit()

print("✅ Relaciones insertadas correctamente.")

✅ Relaciones insertadas correctamente.


In [27]:
# Cerramos la conexión con la base de datos y sql

cnx.close()